## Chapter 5, Series Solution of First-Order and Second-Order Near Equations

## 5.1 Review of Power Series

복습 Chapter이므로 skip하겠습니다.

## 5.2 Power Series Solution of First-Order Differential Equations

이때까지와는 다르게 Power Series Solution를 구하는 단원입니다. Sympy에서는 멱급수에 해당하는 풀이법을 제공하지 않습니다. 그래서 마치 손으로 풀었듯이 알고리즘을 직접 구현하여 상수항부터 대입하는 방법으로 해결했습니다. 이는 일반화 된 알고리즘이라고 보긴 힘들 것 같습니다. 예외적인 처리를 조금 더 해줘야 할 것 같습니다.

In [7]:
import sympy as sy
from sympy import init_printing

init_printing(use_latex='mathjax')
x = sy.symbols('x')
y = sy.Function('y')(x)

max_order = 10 #최고 차항 차수
alist = list() #미지수 리스트

constants = list()

y = 0 #풀어낼 답 y

def make_power_series(y, max_order):
    for i in range(max_order):
        c = sy.symbols('a' + str(i))
        alist.append(c)
        y += c * x**i
    return y

def solve(y, eq):
    index = 1

    # 상수항의 계수를 모으는 부분 추가
    coeff_lhs = eq_simple.lhs.coeff(x,0)
    coeff_rhs = eq_simple.rhs.coeff(x,0)
    constant = sy.solve(sy.Eq(coeff_lhs, coeff_rhs), alist[index])
    constants.append(constant[0])
    index+=1

    for i in range(1, max_order - 1):
        coeff_lhs = sy.collect(eq_simple.lhs, x**i).coeff(x**i)
        coeff_rhs = sy.collect(eq_simple.rhs, x**i).coeff(x**i)
        small_eq = sy.Eq(coeff_lhs, coeff_rhs)
        small_eq.subs(alist[i], constants[i])

        coeff_lhs_substituted = coeff_lhs.subs(alist[i], constants[i])
        small_eq = sy.Eq(coeff_lhs_substituted, coeff_rhs)

        # 조건식을 이용하여 a_n을 a_0로 표현
        condition = small_eq.subs(alist[i], constants[0])
        constant_solution = sy.solve(condition, alist[index])
        constants.append(constant_solution[0])
        index+=1
    
    
    for i in range(0, max_order):
        y = y.subs(alist[i], constants[i])
    #각각의 멱급수를 a_0로 표현해서 반환
    
    return y

y = make_power_series(y, max_order)

constants.append(alist[0])## a_0로 표현될 계수 리스트

eq = sy.Eq((x - 3) * sy.diff(y, x, 1) + 2 * y, 0)
eq_simple = eq.simplify()

y = solve(y, eq_simple)
print("Solution : ")
display(y)


Solution : 


       9       8         7         6         5         4         3       2    
10⋅a₀⋅x    a₀⋅x    8⋅a₀⋅x    7⋅a₀⋅x    2⋅a₀⋅x    5⋅a₀⋅x    4⋅a₀⋅x    a₀⋅x    2
──────── + ───── + ─────── + ─────── + ─────── + ─────── + ─────── + ───── + ─
 19683      729      2187      729        81        81        27       3      

          
⋅a₀⋅x     
───── + a₀
 3        

## 5.4 Series Solutions near an Ordinary Point II

수렴 범위에 대해 다룹니다. 구현할 내용은 없어서 skip하도록 하겠습니다.

## 5.5 Euler Equations, Regular Singular Points

앞에서 다룬 내용 중 $$ y = x^r $$, $$x^2y'' + axy' + by = 0$$
꼴인 특정한 미분방정식 식을 Euler Equations이라고 합니다. 이 식에 있어서는 쉽게 풀 수 있는 방법이 존재하고 그 방법론에 대해 구현해보겠습니다.

전체적인 코드는 특정방정식에서 썼던것 과 비슷합니다.

In [6]:
import sympy as sy
from sympy import init_printing

init_printing(use_latex='mathjax')

x = sy.symbols('x')
f = sy.Function('f')
y = f(x)
r = sy.symbols('r')
c1 = sy.symbols('c1')
c2 = sy.symbols('c2')

a = 2
b = 3
c = -1

eq = sy.Eq(a*x**2*sy.diff(y, x, 2) + b*x * sy.diff(y, x, 1) + c*y, 0)
characteristic_eq = a*r*(r-1) + b*r + c

solutions = sy.solve(characteristic_eq, r)

sol, basis1, basis2 = 0, 0 , 0 #solution, basis1, basis2 선언

if len(solutions) == 1: #중근
    basis1 = x**solutions[0]
    basis2 = x**solutions[0] * sy.log(x)
else: 
    coeff_1 = abs(solutions[0].as_real_imag()[1])
    coeff_2 = solutions[0].as_real_imag()[0]
    
    if coeff_1 != 0: #허수부가 있을 경우
        if coeff_2 != 0:
            basis1 = x**coeff_2* sy.cos(sy.log(coeff_1*x))
            basis2 = x**coeff_2* sy.sin(sy.log(coeff_1*x))
        else:
            basis1 = sy.cos(coeff_1*t)
            basis2 = sy.sin(coeff_1*t)
    else: #허수부가 없을 경우
        basis1 = x**solutions[0]
        basis2 = x**solutions[1]

sol = c1*basis1 + c2*basis2

display(sol)


c₁        
── + c₂⋅√x
x         

## 5.6 Series Solution near a Regular Singular Points, Part 

이제는 Regular Singular Points 근처에서의 멱급수 해를 구합니다. 역시나 책에 있는 방법을 통해 직접 알고리즘을 구해서 구현하고 앞에서 했던 것과 굉장히 흡사한 방법입니다.

In [88]:
import sympy as sy
from sympy import init_printing

init_printing(use_latex='mathjax')
x = sy.symbols('x')
y = sy.Function('y')(x)
r = sy.symbols('r')

max_order = 5 #최고 차항 차수
alist = list() #미지수 리스트

constants = list()

y = 0 #풀어낼 답 y

def make_power_series(y, max_order):
    for i in range(max_order):
        c = sy.symbols('a' + str(i))
        alist.append(c)
        y += c * x**(i+r)
    
    return y

def solve(y, eq):
    index = 1
    # 상수항이 아니라 x^r의 계수를 모으는 부분으로 수정
    coeff_lhs = sy.collect(eq.lhs, x**r).coeff(x**r)#x^r에서 계수를 때내서
    coeff_rhs = sy.collect(eq.rhs, x**r).coeff(x**r)
    coeff_lhs_real = coeff_lhs.coeff(x, 0)#상수항을 모은 후
    characteristic_answer = sy.solve(sy.Eq(coeff_lhs_real, coeff_rhs), r)#특성방정식 계산
    
    eq1 = eq.subs(r, characteristic_answer[0])#식에 r1값 대입
    eq2 = eq.subs(r, characteristic_answer[1])#식에 r2값 대입
    
    r1 = characteristic_answer[0]
    eq_1_lhs = sy.collect(eq1.lhs, x**(r1)).coeff(x**(r1)) # x^r 계수를 때냄
    
    for i in range(1, max_order):
        coeff_lhs = sy.collect(eq_1_lhs, x**i).coeff(x, i) 
        coeff_rhs = sy.collect(eq1.rhs, x**i).coeff(x**i)
        coeff_lhs_substituted = coeff_lhs.subs(alist[i-1], constants[i-1])
        small_eq = sy.Eq(coeff_lhs_substituted, coeff_rhs)
        
        # 조건식을 이용하여 a_n을 a_0로 표현
        condition = small_eq.subs(alist[i-1], constants[0])
        constant_solution = sy.solve(condition, alist[index])
        constants.append(constant_solution[0])
        index+=1
    
    y1 = y.copy()
    for i in range(0, max_order):
        y1 = y1.subs(alist[i], constants[i])
    y1= y1.subs(r, r1)
    #각각의 멱급수를 a_0로 표현해서 반환
    
    index = 1
    r2 = characteristic_answer[1]
    eq_2_lhs = sy.collect(eq2.lhs, x**(r2)).coeff(x**(r2)) # x^r 계수를 때냄
    
    for i in range(1, max_order):
        coeff_lhs = sy.collect(eq_2_lhs, x**i).coeff(x, i) 
        coeff_rhs = sy.collect(eq2.rhs, x**i).coeff(x**i)
        coeff_lhs_substituted = coeff_lhs.subs(alist[i-1], constants[i-1])
        small_eq = sy.Eq(coeff_lhs_substituted, coeff_rhs)
        
        # 조건식을 이용하여 a_n을 a_0로 표현
        condition = small_eq.subs(alist[i-1], constants[0])
        constant_solution = sy.solve(condition, alist[index])
        constants.append(constant_solution[0])
        index+=1
    
    y2 = y.copy()
    for i in range(0, max_order):
        y2 = y2.subs(alist[i], constants[i])
    #각각의 멱급수를 a_0로 표현해서 반환
    y2 = y2.subs(r, r2)
    
    print("Solution : \n")

    print(f"r1 : {characteristic_answer[0]}\n\n y1 = ")
    display(y1)
    
    print(f"r2 : {characteristic_answer[1]}\n\n y2 = ")
    display(y2)
    
y = make_power_series(y, max_order)

constants.append(alist[0])## a_0로 표현될 계수 리스트

eq = sy.Eq(2*x**2 * sy.diff(y, x, 2) -x*sy.diff(y, x, 1) + (1+x) * y, 0)
eq_simple = eq.simplify()

solve(y, eq_simple)



Solution : 

r1 : 1/2

 y1 = 


    9/2       7/2       5/2                  
a₀⋅x      a₀⋅x      a₀⋅x          3/2        
─────── - ─────── + ─────── - a₀⋅x    + a₀⋅√x
  2520       90        6                     

r2 : 1

 y2 = 


    5       4       3               
a₀⋅x    a₀⋅x    a₀⋅x        2       
───── - ───── + ───── - a₀⋅x  + a₀⋅x
 2520     90      6                 

## 5.7 Series Solution near a Regular Singular Points, Part II

수렴 범위에 대해 다룹니다. 구현할 내용은 없어서 skip하도록 하겠습니다.

## 5.8 Bessel's Equation

이 Chapter에서는 Bessel Equation에 대해서 다룹니다. 

그 중 Thm 5.8.1 ~ 5.8.6까지 나오는 Bessel Function에 대한 성질을 수치적으로 다뤄보겠습니다.

In [31]:
import sympy as sy
from sympy import init_printing
from IPython.display import clear_output
x = sy.symbols('x')
n = sy.symbols('n')

init_printing(use_latex='mathjax')

def bessel(n, x, min_order, max_order):
    y = 0
    for r in range(min_order, max_order):
        y += (-1)**r/(sy.factorial(r) * sy.factorial(n+r))*(x/2)**(n+2*r)
    return y

"""
Ex 5.8.1
 J_n(-x) = -1^n*J_n(x)
"""

eq1 = (bessel(n, -x, 0, 5))
eq2 = ((-1)**n * bessel(n,x,0,5))

for i in range(1, 5000):
    if eq1.subs(n, i)- eq2.subs(n, i) != 0:
        display(eq1.subs(n, i)- eq2.subs(n, i)) # 다르면 값을 출력하도록 만듭니다.
    else:
        print(f"{i}번째 okay")
        clear_output(wait=True)
print("5000까지 확인 결과 양 식은 모두 같습니다.")

5000까지 확인 결과 양 식은 모두 같습니다.


In [61]:
import sympy as sy
from sympy import init_printing
from IPython.display import clear_output

x = sy.symbols('x')
n = sy.symbols('n')

init_printing(use_latex='mathjax')

def bessel(n, x, min_order, max_order):
    y = 0
    for r in range(min_order, max_order):
        y += (-1)**r/(sy.factorial(r) * sy.factorial(n+r))*(x/2)**(n+2*r)
    return y


"""
Ex 5.8.2
 2J_n'(x) = J_n-1(x) -  J_n+1(x) 
"""

eq1 = 2 * sy.diff(bessel(n, x, 0, 5), x)
eq2 = bessel(n-1, x, 0, 5) - bessel(n+1, x, 0, 4)

eq1 = sy.simplify(eq1)
eq2 = sy.simplify(eq2)

display(eq1)
display(eq2)

for i in range(1, 5000):
    if eq1.subs(n, i)- eq2.subs(n, i) != 0:
        display(eq1.subs(n, i)- eq2.subs(n, i)) # 다르면 값을 출력하도록 만듭니다.
    else:
        print(f"{i}번째 okay")
        clear_output(wait=True)
print("5000까지 확인 결과 양 식은 모두 같습니다.")


5000까지 확인 결과 양 식은 모두 같습니다.


In [9]:
import sympy as sy
from sympy import init_printing
from IPython.display import clear_output

x = sy.symbols('x')
n = sy.symbols('n')

init_printing(use_latex='mathjax')

def bessel(n, x, min_order, max_order):
    y = 0
    for r in range(min_order, max_order):
        y += (-1)**r/(sy.factorial(r) * sy.factorial(n+r))*(x/2)**(n+2*r)
    return y

"""
Ex 5.8.2 2nJ_n(x) = x(J_n-1(x) +  J_n+1(x)) 
"""

eq1 = 2*n * bessel(n, x, 0, 5)
eq2 = x*( bessel(n-1, x, 0, 5) + bessel(n+1, x, 0, 4))

eq1 = sy.simplify(eq1)
eq2 = sy.simplify(eq2)

for i in range(1, 5000):
    if eq1.subs(n, i)- eq2.subs(n, i) != 0:
        display(eq1.subs(n, i)- eq2.subs(n, i)) # 다르면 값을 출력하도록 만듭니다.
    else:
        print(f"{i}번째 okay")
        clear_output(wait=True)
print("5000까지 확인 결과 양 식은 모두 같습니다.")

5000까지 확인 결과 양 식은 모두 같습니다.


In [10]:
"""
Thm 5.8.11 d/dx{x^n*J_n(x)} = x^n*J_n-l(x)
"""
import sympy as sy
from sympy import init_printing
from IPython.display import clear_output

x = sy.symbols('x')
n = sy.symbols('n')

init_printing(use_latex='mathjax')

def bessel(n, x, min_order, max_order):
    y = 0
    for r in range(min_order, max_order):
        y += (-1)**r/(sy.factorial(r) * sy.factorial(n+r))*(x/2)**(n+2*r)
    return y


eq1 = sy.diff(x**n * bessel(n, x, 0, 5), x)
eq2 = x**n*bessel(n-1, x, 0, 5)

eq1 = sy.simplify(eq1)
eq2 = sy.simplify(eq2)

display(eq1)
display(eq2)

for i in range(1, 5000):
    if eq1.subs(n, i)- eq2.subs(n, i) != 0:
        display(eq1.subs(n, i)- eq2.subs(n, i)) # 다르면 값을 출력하도록 만듭니다.
    else:
        print(f"{i}번째 okay")
        clear_output(wait=True)
print("5000까지 확인 결과 양 식은 모두 같습니다.")


5000까지 확인 결과 양 식은 모두 같습니다.


In [11]:
"""
Thm 5.8.2 d/dx{x^-n*J_n(x)} = -x^-n*J_n+l(x)
"""
import sympy as sy
from sympy import init_printing
from IPython.display import clear_output

x = sy.symbols('x')
n = sy.symbols('n')

init_printing(use_latex='mathjax')

def bessel(n, x, min_order, max_order):
    y = 0
    for r in range(min_order, max_order):
        y += (-1)**r/(sy.factorial(r) * sy.factorial(n+r))*(x/2)**(n+2*r)
    return y


eq1 = sy.diff(x**(-n) * bessel(n, x, 0, 5), x)
eq2 = -1*x**(-n)*bessel(n+1, x, 0, 4)

eq1 = sy.simplify(eq1)
eq2 = sy.simplify(eq2)

for i in range(1, 5000):
    if eq1.subs(n, i)- eq2.subs(n, i) != 0:
        display(eq1.subs(n, i)- eq2.subs(n, i)) # 다르면 값을 출력하도록 만듭니다.
    else:
        print(f"{i}번째 okay")
        clear_output(wait=True)
print("5000까지 확인 결과 양 식은 모두 같습니다.")

5000까지 확인 결과 양 식은 모두 같습니다.


In [14]:
"""
Thm 5.8.3 J_n(x)' = J_n-1(x) - (n/x)J_n(x)
"""
import sympy as sy
from sympy import init_printing
from IPython.display import clear_output

x = sy.symbols('x')
n = sy.symbols('n')

init_printing(use_latex='mathjax')

def bessel(n, x, min_order, max_order):
    y = 0
    for r in range(min_order, max_order):
        y += (-1)**r/(sy.factorial(r) * sy.factorial(n+r))*(x/2)**(n+2*r)
    return y


eq1 = sy.diff(bessel(n, x, 0, 5), x)
eq2 = bessel(n-1, x, 0, 5) - (n/x)*bessel(n, x, 0, 5)

eq1 = sy.simplify(eq1)
eq2 = sy.simplify(eq2)

for i in range(1, 5000):
    if eq1.subs(n, i)- eq2.subs(n, i) != 0:
        display(eq1.subs(n, i)- eq2.subs(n, i)) # 다르면 값을 출력하도록 만듭니다.
    else:
        print(f"{i}번째 okay")
        clear_output(wait=True)
print("5000까지 확인 결과 양 식은 모두 같습니다.")

5000까지 확인 결과 양 식은 모두 같습니다.


In [16]:
"""
Thm 5.8.4 J_n(x)' = (n/x)J_n(x) - J_n+1(x)
"""
import sympy as sy
from sympy import init_printing
from IPython.display import clear_output

x = sy.symbols('x')
n = sy.symbols('n')

init_printing(use_latex='mathjax')

def bessel(n, x, min_order, max_order):
    y = 0
    for r in range(min_order, max_order):
        y += (-1)**r/(sy.factorial(r) * sy.factorial(n+r))*(x/2)**(n+2*r)
    return y


eq1 = sy.diff(bessel(n, x, 0, 5), x)
eq2 = (n/x)*bessel(n, x, 0, 5) -  bessel(n+1, x, 0, 4)

eq1 = sy.simplify(eq1)
eq2 = sy.simplify(eq2)

for i in range(1, 5000):
    if eq1.subs(n, i)- eq2.subs(n, i) != 0:
        display(eq1.subs(n, i)- eq2.subs(n, i)) # 다르면 값을 출력하도록 만듭니다.
    else:
        print(f"{i}번째 okay")
        clear_output(wait=True)
print("5000까지 확인 결과 양 식은 모두 같습니다.")

5000까지 확인 결과 양 식은 모두 같습니다.


In [39]:
"""
Thm 5.8.5 J_n(x)' = 1/2(J_n-1(x) - J_n+1(x))
"""
import sympy as sy
from sympy import init_printing
from IPython.display import clear_output

x = sy.symbols('x')
n = sy.symbols('n')

init_printing(use_latex='mathjax')

def bessel(n, x, min_order, max_order):
    y = 0
    for r in range(min_order, max_order):
        y += (-1)**r/(sy.factorial(r) * sy.factorial(n+r))*(x/2)**(n+2*r)
    return y


eq1 = 1.0 * sy.diff(bessel(n, x, 0, 5), x)
eq2 = (1/2)*(bessel(n-1, x, 0, 5) -  bessel(n+1, x, 0, 4))

count = 0
for i in range(1, 5000):
    if eq1.subs(n, i)- eq2.subs(n, i) != 0:
         count+=1 # 다르면 다른 개수 추가
    else:
        print(f"{i}번째 okay")
        clear_output(wait=True)
print(f"5000까지 확인 결과 {5000 - count}개는 양 식이 완벽히 같습니다.")

"""
부동 소수점 문제로 양 식이 같아도 같다고 나오지 않습니다.
다른 방법이 떠오르지 않아 조작없이 그대로 추가했습니다.
"""


5000까지 확인 결과 547개는 양 식이 완벽히 같습니다.


'\n부동 소수점 문제로 양 식이 같아도 같다고 나오지 않습니다.\n다른 방법이 떠오르지 않아 조작없이 그대로 추가했습니다.\n'

In [13]:
"""
Thm 5.8.6 J_n(x)' = 1/2(J_n-1(x) - J_n+1(x))
"""
import sympy as sy
from sympy import init_printing
from IPython.display import clear_output

x = sy.symbols('x')
n = sy.symbols('n')

init_printing(use_latex='mathjax')

def bessel(n, x, min_order, max_order):
    y = 0
    for r in range(min_order, max_order):
        y += (-1)**r/(sy.factorial(r) * sy.factorial(n+r))*(x/2)**(n+2*r)
    return y


eq1 = bessel(n-1, x, 0, 5) +  bessel(n+1, x, 0, 4)
eq2 = (2*n/x)*(bessel(n, x, 0, 5))

eq1 = sy.simplify(eq1)
eq2 = sy.simplify(eq2)

display(eq1)
display(eq2)


for i in range(1, 5000):
    if eq1.subs(n, i)- eq2.subs(n, i) != 0:
        display(eq1.subs(n, i)- eq2.subs(n, i)) # 다르면 값을 출력하도록 만듭니다.
    else:
        print(f"{i}번째 okay")
        clear_output(wait=True)
print("5000까지 확인 결과 양 식은 모두 같습니다.")

5000까지 확인 결과 양 식은 모두 같습니다.


뒤에 부분이 더 남았지만, 부동소수점 문제로 어느 정도 오차를 허용해야할지 모르겠어서, 여기까지만 수치적 증명을 남깁니다.